In [1]:
import pandas as pd
import re
import nltk   
from urllib.request import urlopen
from nltk.stem import SnowballStemmer

In [2]:
# прочитаем основные данные
table = pd.read_csv('dataset_20211126.csv', sep=';', encoding='cp1251')

In [3]:
table.head(5)

,TNVED,OPISANIE,DATA
0,9025,"ЧАСТИ И ПРИНАДЛЕЖНОСТИ АППАРАТУРЫ ПОЗ.9025, НЕ...",07.09.2021
1,8504,"ТРАНСФОРМАТОРЫ, МОЩНОСТЬЮ БОЛЕЕ 1 КВА, НО НЕ Б...",07.09.2021
2,8481,"КЛАПАНЫ ЗАПОРНЫЕ ИЗ СТАЛИ, НЕ ВОЕННОГО НАЗНАЧЕ...",07.09.2021
3,8531,УСТРОЙСТВА СИГНАЛИЗАЦИОННЫЕ ОХРАННЫЕ И УСТРОЙС...,07.09.2021
4,3402,"ВЕЩЕСТВА ПОВЕРХНОСТНО-АКТИВНЫЕ ОРГАНИЧЕСКИЕ, А...",02.09.2021


In [4]:
shape_full = table.shape
print("размер исходного датасета: ", shape_full)

размер исходного датасета:  (4177899, 3)


In [5]:
# дата нам не понадобятся
del table['DATA']

In [6]:
# посмотрим сколько уникальных TNVED кодов
table.groupby(['TNVED']).count().describe()

,OPISANIE
count,1192.000000
mean,3504.948826
std,11335.752609
min,1.000000
25%,94.000000
50%,508.000000
75%,2101.250000
max,208058.000000


In [7]:
# видно что уникальных TNVED кодов в базе 1192

In [8]:
# проверим вдруг есть прям одинаковые строки, для этого сгруппируем по описание
table.groupby(['OPISANIE']).count().describe()

,TNVED
count,1.579686e+06
mean,2.644765e+00
std,1.698474e+01
min,1.000000e+00
25%,1.000000e+00
50%,1.000000e+00
75%,1.000000e+00
max,6.363000e+03


In [9]:
# видно что много повторяющихся описаний, так как уникальных описаний в разы меньше, чем данных в датасете
# удалим, повторяющиеся данные

In [10]:
table = table.drop_duplicates(keep='last')
shape_after_drop_duplicate = table.shape
print("размер датасета после удаления дубликатов: ", shape_after_drop_duplicate)

размер датасета после удаления дубликатов:  (1598797, 2)


In [11]:
# сохраним 
table.to_csv('dataset_without_duplicate.csv')

In [12]:
table.head(5)

,TNVED,OPISANIE
1,8504,"ТРАНСФОРМАТОРЫ, МОЩНОСТЬЮ БОЛЕЕ 1 КВА, НО НЕ Б..."
7,9004,"ОЧКИ КОРРИГИРУЮЩИЕ, С ЛИНЗАМИ ИЗ ПЛАСТМАССЫ, В..."
8,9019,"РОЛИК МАССАЖНЫЙ ДЛЯ ЛИЦА, ИЗ АГЛОМЕРИРОВАННОГО..."
9,9405,"СВЕТИЛЬНИКИ УЛИЧНЫЕ, ИЗ МЕТАЛЛА И ПЛАСТМАССЫ; ..."
10,9405,"СВЕТИЛЬНИКИ-ФОНАРИ УЛИЧНЫЕ, ТРЕКОВЫЕ, ДЛЯ ПОДС..."


In [13]:
# как выглядит описание
table.iloc[30]

TNVED                                                    7318
OPISANIE    НОВЫЕ ЗАПАСНЫЕ ЧАСТИ ДЛЯ ФРОНТАЛЬНЫХ КОЛЕСНЫХ ...
Name: 77, dtype: object

In [14]:
def clean(text):
    text = text.lower()
    
    text = text + ' '
    text = text.replace(' не ', ' не')
    text = text.replace(' без ', ' без')
    text = text.replace(' назначения', 'назначения')
    
    text = re.sub('\W',' ', text) # Любая не-буква, не-цифра и не подчёркивание
    
    text = text.replace(' и ',' ')
    text = text.replace(' но ',' ')
    text = text.replace(' для ',' ')
    text = text.replace(' недля ',' недля')
    text = text.replace(' неявляется ',' неявляется')
    text = text.replace(' неявляются ',' неявляются')
    text = text.replace(' несодержит ',' несодержит')
    text = text.replace(' несодержат ',' несодержат')
    text = text.replace(' неимеются ',' неимеются')
    text = text.replace(' неимеется ',' неимеется')
    text = text.replace(' неимеют ',' неимеют')
    text = text.replace(' неимеет ',' неимеет')
    text = text.replace(' из ',' ')
    text = text.replace(' поз ',' ')
    text = text.replace(' как ',' ')
    
    text = text.replace(' более ',' ')
    text = text.replace(' менее ',' ')
    text = text.replace(' неболее ',' ')
    text = text.replace(' неменее ',' ')
    text = text.replace(' всего ',' ')
    
#     text = re.sub('.\S',' ', text) # Точка и любой НЕ пробельный символ
#     text = re.sub(',\S',' ', text) # Запятая и любой НЕ пробельный символ
#     text = re.sub(';\S',' ', text) #
    
    text = re.sub(r'[^а-яА-Я\s]+', '', text)
    text = text.replace(' кг\s',' ')
    
    text = re.sub('ква\s',' ', text)
    
    text = re.sub('\s\w\s',' ', text) # Одна буква
    text = re.sub('\s\w\w\s',' ', text) # две буквы
    text = re.sub('\s\w\w\s',' ', text) # две буквы
    text = re.sub('\s\w\w\s',' ', text) # две буквы
    text = re.sub('\s\w\s',' ', text) # Одна буква

    return text

In [15]:
table['OPISANIE'] = table['OPISANIE'].apply(clean)
table.head()

,TNVED,OPISANIE
1,8504,трансформаторы мощностью нелом электроо...
7,9004,очки корригирующие линзами пластмассы полиме...
8,9019,ролик массажный лица агломерированного искус...
9,9405,светильники уличные металла пластмассы предн...
10,9405,светильники фонари уличные трековые подсветк...


In [16]:
table = table.drop_duplicates(keep='last')
shape_after_drop_duplicate2 = table.shape
print('изначальный: ', shape_full)
print('после первой очистки: ', shape_after_drop_duplicate)
print('после второй очистки: ', shape_after_drop_duplicate2)


изначальный:  (4177899, 3)
после первой очистки:  (1598797, 2)
после второй очистки:  (1242807, 2)


In [17]:
stemmer_ru = SnowballStemmer("russian")
# нормализуем слова     
def norm_str(text):
    text = text.split()
    temp = [stemmer_ru.stem(word) for word in text]
    str_norm = ' '.join(temp)
    return str_norm

In [18]:
# нормализуем слова
temp = pd.DataFrame()
temp['OPISANIE'] = table['OPISANIE'].apply(norm_str)

In [19]:
table['OPISANIE'] = temp['OPISANIE']
table.head()

,TNVED,OPISANIE
1,8504,трансформатор мощност нел электрооборудован
8,9019,ролик массажн лиц агломерирова искусствен камн...
9,9405,светильник уличн металл пластмасс предназначен...
10,9405,светильник фонар уличн треков подсветк рекламн...
13,1207,проч сем плод проч масличн культур


In [20]:
table = table.drop_duplicates(keep='last')
table.shape

(1086705, 2)

In [21]:
shape_after_drop_duplicate3 = table.shape
print('изначальный: ', shape_full)
print('после первой очистки: ', shape_after_drop_duplicate)
print('после второй очистки: ', shape_after_drop_duplicate2)
print('после третьей очистки (после стеммизации): ', shape_after_drop_duplicate3)

изначальный:  (4177899, 3)
после первой очистки:  (1598797, 2)
после второй очистки:  (1242807, 2)
после третьей очистки (после стеммизации):  (1086705, 2)


In [24]:
# Теперь сделаем балансиравку классов, для этого посчитаем сколько в каждом классе данных
table['len_opisanie'] = table['OPISANIE'].apply(len)
table.head()

,TNVED,OPISANIE,len_opisanie
8,9019,ролик массажн лиц агломерирова искусствен камн...,92
9,9405,светильник уличн металл пластмасс предназначен...,120
10,9405,светильник фонар уличн треков подсветк рекламн...,164
13,1207,проч сем плод проч масличн культур,34
15,8208,нож рубительн машин,19


In [25]:
table = table.sort_values(by='len_opisanie', ascending=False)
table.head()

,TNVED,OPISANIE,len_opisanie
3646289,8431,част бульдозер экскаватор грейдер стойк рыхлит...,214
661713,1704,конфет мармелад ассорт фруктов вкус соста сиро...,213
3530959,8516,электрооборудован обогрев пространств встроен ...,210
1844688,3004,лекарствен средств белосалик лосьон раствор на...,210
3371156,8516,тепловентилятор бытов напольн электрическ отоп...,210


In [26]:
table['cumcount'] = table.groupby("TNVED").cumcount()
table.head()

,TNVED,OPISANIE,len_opisanie,cumcount
3646289,8431,част бульдозер экскаватор грейдер стойк рыхлит...,214,0
661713,1704,конфет мармелад ассорт фруктов вкус соста сиро...,213,0
3530959,8516,электрооборудован обогрев пространств встроен ...,210,0
1844688,3004,лекарствен средств белосалик лосьон раствор на...,210,0
3371156,8516,тепловентилятор бытов напольн электрическ отоп...,210,1


In [27]:
# если в классе больше 200 примеров, то оставляем 200 самых длинных описаний
table = table.query("cumcount < 200")
table.head()

,TNVED,OPISANIE,len_opisanie,cumcount
3646289,8431,част бульдозер экскаватор грейдер стойк рыхлит...,214,0
661713,1704,конфет мармелад ассорт фруктов вкус соста сиро...,213,0
3530959,8516,электрооборудован обогрев пространств встроен ...,210,0
1844688,3004,лекарствен средств белосалик лосьон раствор на...,210,0
3371156,8516,тепловентилятор бытов напольн электрическ отоп...,210,1


In [28]:
table.groupby(['TNVED']).count().describe()

,OPISANIE,len_opisanie,cumcount
count,1192.000000,1192.000000,1192.000000
mean,129.191275,129.191275,129.191275
std,79.152460,79.152460,79.152460
min,1.000000,1.000000,1.000000
25%,47.000000,47.000000,47.000000
50%,189.000000,189.000000,189.000000
75%,200.000000,200.000000,200.000000
max,200.000000,200.000000,200.000000


In [29]:
shape_after_balanced_classes = table.shape
print('изначальный: ', shape_full)
print('после первой очистки: ', shape_after_drop_duplicate)
print('после второй очистки: ', shape_after_drop_duplicate2)
print('после третьей очистки (после стеммизации): ', shape_after_drop_duplicate3)
print('после четвертой очистки (после балансировки классов): ', shape_after_balanced_classes)

изначальный:  (4177899, 3)
после первой очистки:  (1598797, 2)
после второй очистки:  (1242807, 2)
после третьей очистки (после стеммизации):  (1086705, 2)
после четвертой очистки (после балансировки классов):  (153996, 4)


In [30]:
del table['len_opisanie']
del table['cumcount']
table.head()

,TNVED,OPISANIE
3646289,8431,част бульдозер экскаватор грейдер стойк рыхлит...
661713,1704,конфет мармелад ассорт фруктов вкус соста сиро...
3530959,8516,электрооборудован обогрев пространств встроен ...
1844688,3004,лекарствен средств белосалик лосьон раствор на...
3371156,8516,тепловентилятор бытов напольн электрическ отоп...


In [46]:
table.to_csv('dataset_clean_balanced_classes.csv')

In [44]:
table[153976 : 153996]

,TNVED,OPISANIE
1295879,5515,ткан
2875524,8522,част
470342,1214,сен
1152238,9005,луп
3769971,8417,печ
1210427,2801,йод
2254117,209,сал
4063016,9604,сит
4076323,407,яйц
1685678,9104,час


## перейдем к векторизации описания

In [51]:
from stop_words import get_stop_words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [52]:
# посмотрим на стоп слова
stop_words_ru = get_stop_words('ru')
print(stop_words_ru)

['а', 'в', 'г', 'е', 'ж', 'и', 'к', 'м', 'о', 'с', 'т', 'у', 'я', 'бы', 'во', 'вы', 'да', 'до', 'ее', 'ей', 'ею', 'её', 'же', 'за', 'из', 'им', 'их', 'ли', 'мы', 'на', 'не', 'ни', 'но', 'ну', 'нх', 'об', 'он', 'от', 'по', 'со', 'та', 'те', 'то', 'ту', 'ты', 'уж', 'без', 'был', 'вам', 'вас', 'ваш', 'вон', 'вот', 'все', 'всю', 'вся', 'всё', 'где', 'год', 'два', 'две', 'дел', 'для', 'его', 'ему', 'еще', 'ещё', 'или', 'ими', 'имя', 'как', 'кем', 'ком', 'кто', 'лет', 'мне', 'мог', 'мож', 'мои', 'мой', 'мор', 'моя', 'моё', 'над', 'нам', 'нас', 'наш', 'нее', 'ней', 'нем', 'нет', 'нею', 'неё', 'них', 'оба', 'она', 'они', 'оно', 'под', 'пор', 'при', 'про', 'раз', 'сам', 'сих', 'так', 'там', 'тем', 'тех', 'том', 'тот', 'тою', 'три', 'тут', 'уже', 'чем', 'что', 'эта', 'эти', 'это', 'эту', 'алло', 'буду', 'будь', 'бывь', 'была', 'были', 'было', 'быть', 'вами', 'ваша', 'ваше', 'ваши', 'ведь', 'весь', 'вниз', 'всем', 'всех', 'всею', 'года', 'году', 'даже', 'двух', 'день', 'если', 'есть', 'зато', 'ко

In [53]:
stop_words_ru = stop_words_ru + ['предназначены' + 'является' + 'предназначенны' + 'применяется' + 'изготовлен']
# нормализуем стоп слова тоже
stop_words_all = [stemmer_ru.stem(word) for word in stop_words_ru]

In [55]:
text = table['OPISANIE']
countVectorizer = CountVectorizer(stop_words=stop_words_all, min_df=15).fit(text)
vocabulary_countVectorizer = countVectorizer.get_feature_names()
print('Размер словаря:', len(vocabulary_countVectorizer))

Размер словаря: 10289


In [56]:
X_countVectorizer = countVectorizer.transform(text)
top_tokens, _ = zip(*sorted(zip(vocabulary_countVectorizer, X_countVectorizer.sum(axis=0).getA1()), key=lambda x: x[1], reverse=True)[:100])
print('Top-100 слов:', top_tokens)

Top-100 слов: ('предназнач', 'издел', 'производств', 'упаковк', 'качеств', 'вид', 'вес', 'использ', 'средств', 'использован', 'изготовлен', 'промышлен', 'продаж', 'спирт', 'материал', 'машин', 'материа', 'представля', 'товар', 'поставля', 'упакова', 'применя', 'содержан', 'размер', 'марк', 'комплект', 'соста', 'оборудован', 'пищев', 'предназначен', 'основ', 'розничн', 'металл', 'картон', 'покрыт', 'нит', 'цвет', 'веществ', 'арт', 'синтетическ', 'форм', 'коробк', 'текстильн', 'продукт', 'химическ', 'цел', 'обработк', 'ткан', 'применен', 'работ', 'образц', 'поддон', 'нетт', 'двигател', 'элемент', 'явля', 'различн', 'устройств', 'масс', 'изготовл', 'трикотажн', 'проведен', 'систем', 'рулон', 'безсодержан', 'лабораторн', 'взросл', 'этилов', 'расфасова', 'набор', 'техническ', 'употреблен', 'продукц', 'бел', 'невоенногоназначен', 'проч', 'ремонт', 'тип', 'мешк', 'электрическ', 'сыр', 'нов', 'вод', 'получен', 'металлическ', 'полимерн', 'содержа', 'номер', 'стекл', 'масл', 'запасн', 'метод', '

In [57]:
X_countVectorizer = countVectorizer.transform(text)
top_tokens, _ = zip(*sorted(zip(vocabulary_countVectorizer, X_countVectorizer.sum(axis=0).getA1()), key=lambda x: x[1], reverse=True))
print('Top-100 слов самых не популярных:', top_tokens[len(top_tokens) - 100: ])

Top-100 слов самых не популярных: ('несодержитсвободн', 'нефтепровод', 'нефтепромыслов', 'неявляетсярадиационностойк', 'нмд', 'нормализ', 'одноголов', 'оксихлоридн', 'окулировк', 'опечатыван', 'орешек', 'основовязальн', 'отгруж', 'отжимн', 'ощипа', 'передат', 'перекрестн', 'пиросульф', 'пластификац', 'пластов', 'пленкообраз', 'пневмоколесн', 'повседн', 'полуглянцев', 'предсталя', 'прилипан', 'примененя', 'проектн', 'промышленнос', 'пропанов', 'пропионов', 'просеян', 'профессион', 'раздражен', 'распашн', 'реа', 'регулирова', 'репродукц', 'рецепт', 'руллон', 'садк', 'самозатуха', 'сантехник', 'свидетельств', 'свистк', 'секонд', 'серг', 'синев', 'служебн', 'смачива', 'смз', 'смс', 'содержащ', 'соотн', 'спел', 'станкостроен', 'статус', 'стеклотар', 'строител', 'сухогрузн', 'съем', 'татнефт', 'тверден', 'телевиден', 'тенсел', 'теплогенератор', 'термоизоляцион', 'термоусадк', 'террариум', 'технологич', 'тиш', 'токсин', 'траспортировк', 'удон', 'узорчат', 'украин', 'укрепл', 'устройст', 'флор